## 데이터 확인 및 전처리

In [29]:
import os
import numpy as np
import cv2
import copy
import random
import matplotlib.pyplot as plt
from ipywidgets import interact

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.models as models
import torchvision.datasets as Datasets
from torchvision import transforms
from torchsummary import summary

In [2]:
train_path = './data/train'
test_path = './data/test'

In [3]:
train_dataset = torchvision.datasets.ImageFolder(train_path)

In [4]:
train_dataset.classes

['Cat', 'Dog']

In [5]:
train_dataset.class_to_idx

{'Cat': 0, 'Dog': 1}

In [6]:
# image path and classes
train_dataset.imgs

[('./data/train\\Cat\\0.jpg', 0),
 ('./data/train\\Cat\\1.jpg', 0),
 ('./data/train\\Cat\\10.jpg', 0),
 ('./data/train\\Cat\\11.jpg', 0),
 ('./data/train\\Cat\\12.jpg', 0),
 ('./data/train\\Cat\\13.jpg', 0),
 ('./data/train\\Cat\\14.jpg', 0),
 ('./data/train\\Cat\\15.jpg', 0),
 ('./data/train\\Cat\\16.jpg', 0),
 ('./data/train\\Cat\\17.jpg', 0),
 ('./data/train\\Cat\\18.jpg', 0),
 ('./data/train\\Cat\\19.jpg', 0),
 ('./data/train\\Cat\\2.jpg', 0),
 ('./data/train\\Cat\\20.jpg', 0),
 ('./data/train\\Cat\\21.jpg', 0),
 ('./data/train\\Cat\\22.jpg', 0),
 ('./data/train\\Cat\\23.jpg', 0),
 ('./data/train\\Cat\\24.jpg', 0),
 ('./data/train\\Cat\\25.jpg', 0),
 ('./data/train\\Cat\\26.jpg', 0),
 ('./data/train\\Cat\\27.jpg', 0),
 ('./data/train\\Cat\\28.jpg', 0),
 ('./data/train\\Cat\\29.jpg', 0),
 ('./data/train\\Cat\\3.jpg', 0),
 ('./data/train\\Cat\\30.jpg', 0),
 ('./data/train\\Cat\\31.jpg', 0),
 ('./data/train\\Cat\\32.jpg', 0),
 ('./data/train\\Cat\\33.jpg', 0),
 ('./data/train\\Cat\\34

In [7]:
@interact(index=(0, len(train_dataset.imgs)-1))
def image_show(index=0):
    image, label = train_dataset.imgs[index]

    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.title('cat' if label == 0 else 'dog')
    plt.imshow(image)
    plt.show()


interactive(children=(IntSlider(value=0, description='index', max=528), Output()), _dom_classes=('widget-inter…

In [8]:
train_transforms = transforms.Compose([
                                        transforms.Resize((256, 256)),
                                        transforms.RandomRotation(5),
                                        transforms.RandomHorizontalFlip(0.5),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

test_transforms = transforms.Compose([
                                        transforms.Resize((256, 256)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [9]:
train_dataset = torchvision.datasets.ImageFolder(train_path, transform=train_transforms)
test_dataset = torchvision.datasets.ImageFolder(test_path, transform=test_transforms)

In [10]:
val_ratio = 0.9
train_len = int(len(train_dataset) * val_ratio)
val_len = len(train_dataset) - train_len

train_data, val_data = data.random_split(train_dataset, [train_len, val_len])

In [11]:
val_data = copy.deepcopy(val_data)
val_data.dataset.transform = test_transforms

In [12]:
print(len(train_data), len(val_data), len(test_dataset))

476 53 12


In [13]:
BATCH_SIZE =  128
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [14]:
dataloaders = {}

dataloaders['train'] = data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
dataloaders['val'] = data.DataLoader(val_data, shuffle=False, batch_size=BATCH_SIZE)
dataloaders['train'] = data.DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)

## Build Model

- self

In [17]:
from model import VGG, get_vgg_layers

-  features : 모델 유형 정의

In [18]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256,  256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

In [24]:
# class 2 : cat or dog
OUTPUT_DIM = 2

# vgg 11
vgg11_layers  = get_vgg_layers(vgg11_config, batch_norm=True)
model = VGG(vgg11_layers, output_dim=OUTPUT_DIM)
summary(model=model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
       BatchNorm2d-6        [-1, 128, 128, 128]             256
              ReLU-7        [-1, 128, 128, 128]               0
         MaxPool2d-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
      BatchNorm2d-10          [-1, 256, 64, 64]             512
             ReLU-11          [-1, 256, 64, 64]               0
           Conv2d-12          [-1, 256, 64, 64]         590,080
      BatchNorm2d-13          [-1, 256, 64, 64]             512
             ReLU-14          [-1, 256,

In [26]:
# class 2 : cat or dog
OUTPUT_DIM = 2

# vgg 13
vgg13_layers  = get_vgg_layers(vgg13_config, batch_norm=True)
model = VGG(vgg13_layers, output_dim=OUTPUT_DIM)
summary(model=model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
         MaxPool2d-7         [-1, 64, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
       BatchNorm2d-9        [-1, 128, 128, 128]             256
             ReLU-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]         147,584
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
        MaxPool2d-14          [-1, 128,

In [27]:
# class 2 : cat or dog
OUTPUT_DIM = 2

# vgg 16
vgg16_layers  = get_vgg_layers(vgg16_config, batch_norm=True)
model = VGG(vgg16_layers, output_dim=OUTPUT_DIM)
summary(model=model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
         MaxPool2d-7         [-1, 64, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
       BatchNorm2d-9        [-1, 128, 128, 128]             256
             ReLU-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]         147,584
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
        MaxPool2d-14          [-1, 128,

In [28]:
# class 2 : cat or dog
OUTPUT_DIM = 2

# vgg 19
vgg19_layers  = get_vgg_layers(vgg19_config, batch_norm=True)
model = VGG(vgg19_layers, output_dim=OUTPUT_DIM)
summary(model=model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
         MaxPool2d-7         [-1, 64, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
       BatchNorm2d-9        [-1, 128, 128, 128]             256
             ReLU-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]         147,584
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
        MaxPool2d-14          [-1, 128,

- use pretrainde model

In [31]:
pretrained11_model = models.vgg11_bn(pretrained=True)
# pretrained13_model = models.vgg13_bn(pretrained=True)
# pretrained16_model = models.vgg16_bn(pretrained=True)
# pretrained19_model = models.vgg19_bn(pretrained=True)

summary(pretrained11_model, input_size=(3, 256, 256))

c:\Users\baebi\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\baebi\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
       BatchNorm2d-6        [-1, 128, 128, 128]             256
              ReLU-7        [-1, 128, 128, 128]               0
         MaxPool2d-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]         295,168
      BatchNorm2d-10          [-1, 256, 64, 64]             512
             ReLU-11          [-1, 256, 64, 64]               0
           Conv2d-12          [-1, 256, 64, 64]         590,080
      BatchNorm2d-13          [-1, 256, 64, 64]             512
             ReLU-14          [-1, 256,